In [14]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import os
from google.colab import drive
import matplotlib.colors as mcolors
from math import pi

# Montar o Google Drive
drive.mount('/content/drive')

# Definir o caminho para a pasta no Drive
caminho_drive = '/content/drive/My Drive/cochrane-et-al/'  # Substitua 'SuaPasta' pelo nome da sua pasta

# Função para carregar os dados de uma URL pública ou arquivo local
def carregar_dados(sheet_url):
    try:
        df = pd.read_csv(sheet_url)
        return df
    except Exception as e:
        print(f"Erro ao carregar dados: {e}")
        return None

# Função para pré-processar os dados
def pre_processar_dados(df):
    if df is None:
        return None

    # Remover espaços nos nomes das colunas e converter vírgulas para pontos decimais
    df.columns = df.columns.str.strip()
    for col in df.columns:
        df[col] = df[col].apply(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)

    # Converter colunas numéricas corretamente
    numeric_columns = df.columns.drop('Faixa etária')
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

    # Normalizar os dados numéricos usando MinMaxScaler
    scaler = MinMaxScaler()
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

    return df

# Função para salvar gráficos no Google Drive
def salvar_grafico_drive(plot, nome_arquivo, caminho_drive):
    caminho_arquivo = os.path.join(caminho_drive, f"{nome_arquivo}.png")
    plot.savefig(caminho_arquivo, dpi=300, bbox_inches='tight')
    print(f"Gráfico salvo no Drive: {caminho_arquivo}")

# Função para gerar gráficos de regressão
def gerar_grafico_regressao(df, x_col, y_col, ordem, titulo, nome_arquivo):
    plt.figure(figsize=(10, 6))
    # Aplicar a cor preta no fundo
    plt.gca().set_facecolor('black')
    plt.gcf().set_facecolor('black')
    sns.regplot(x=x_col, y=y_col, data=df, order=ordem, scatter_kws={'s': 50}, line_kws={'color': 'red'})
    plt.title(titulo, color='white')
    plt.xlabel(x_col, color='white')
    plt.ylabel(y_col, color='white')
    plt.xticks(color='white')
    plt.yticks(color='white')
    salvar_grafico_drive(plt, nome_arquivo, caminho_drive)
    plt.close()

# Função para realizar o teste t de duas amostras independentes
def teste_t_independente(df, coluna, grupo1, grupo2):
    grupo1_dados = df[df['Faixa etária'] == grupo1][coluna]
    grupo2_dados = df[df['Faixa etária'] == grupo2][coluna]

    t_stat, p_value = stats.ttest_ind(grupo1_dados.dropna(), grupo2_dados.dropna())
    print(f"Teste t para {coluna} entre {grupo1} e {grupo2}: t={t_stat:.4f}, p={p_value:.4f}")
    return t_stat, p_value

# Função para gerar um gráfico de boxplot
def gerar_boxplot(df, x_col, y_col, titulo, nome_arquivo):
    plt.figure(figsize=(10, 6))
    plt.gca().set_facecolor('black')
    plt.gcf().set_facecolor('black')
    sns.boxplot(x=x_col, y=y_col, data=df, palette="viridis")
    plt.title(titulo, color='white')
    plt.xlabel(x_col, color='white')
    plt.ylabel(y_col, color='white')
    plt.xticks(color='white')
    plt.yticks(color='white')
    salvar_grafico_drive(plt, nome_arquivo, caminho_drive)
    plt.close()

# Função para gerar um gráfico radar
def gerar_radar(df, colunas, grupo, titulo, nome_arquivo):
    categorias = list(colunas)
    N = len(categorias)

    # Criar ângulos para o radar plot
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]

    # Iniciar o plot
    plt.figure(figsize=(10, 6))
    plt.gca().set_facecolor('black')
    plt.gcf().set_facecolor('black')
    ax = plt.subplot(111, polar=True)

    for faixa_etaria in df[grupo].unique():
        valores = df[df[grupo] == faixa_etaria][colunas].mean().tolist()
        valores += valores[:1]
        ax.plot(angles, valores, linewidth=2, linestyle='solid', label=faixa_etaria)
        ax.fill(angles, valores, alpha=0.3)

    plt.xticks(angles[:-1], categorias, color='white')
    ax.tick_params(axis='y', colors='white')

    # Cor de fundo preta dentro dos círculos
    ax.set_facecolor('black')

    plt.title(titulo, color='white')
    ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1), labelcolor='white')
    salvar_grafico_drive(plt, nome_arquivo, caminho_drive)
    plt.close()

# Função para gerar um gráfico de violino bipartido
def gerar_violino_bipartido(df, x_col, y_col, titulo, nome_arquivo):
    plt.figure(figsize=(10, 6))
    plt.gca().set_facecolor('black')
    plt.gcf().set_facecolor('black')
    sns.violinplot(x=x_col, y=y_col, data=df, split=True, inner="quart", palette="viridis")
    plt.title(titulo, color='white')
    plt.xlabel(x_col, color='white')
    plt.ylabel(y_col, color='white')
    plt.xticks(color='white')
    plt.yticks(color='white')
    salvar_grafico_drive(plt, nome_arquivo, caminho_drive)
    plt.close()

# URL pública do Google Sheets (exportado como CSV público)
sheet_url = "https://docs.google.com/spreadsheets/d/1ri-y0anPqPxR_PXS7sgL_xtEKnvDdrvLU1PVlTjuY7k/export?format=csv&gid=0"

# Carregar e pré-processar os dados
df = carregar_dados(sheet_url)
df = pre_processar_dados(df)

if df is not None:
    # Gráficos de regressão
    gerar_grafico_regressao(df, 'Campo de visão útil', 'Tarefa de rede de atenção: alerta', 3,
                            'Regressão de Campo de Visão Útil vs Alerta (Ordem 3)', 'regressao_alerta_visao_ordem3')

    gerar_grafico_regressao(df, 'Detecção de mudança', 'Detecção de mudança com distratores', 3,
                            'Regressão de Detecção de Mudança vs Com Distratores (Ordem 3)', 'regressao_mudanca_distratores_ordem3')

    gerar_grafico_regressao(df, 'Blocos de Corsi', 'Rastreio de múltiplos objetos', 2,
                            'Regressão de Blocos de Corsi vs Rastreio de Objetos (Ordem 2)', 'regressao_corsi_rastreio_ordem2')

    # Gráfico de boxplot
    gerar_boxplot(df, 'Faixa etária', 'Detecção de mudança',
                  'Boxplot de Detecção de Mudança por Faixa Etária', 'boxplot_faixa_mudanca')

    # Gráfico radar
    gerar_radar(df, ['Campo de visão útil', 'Tarefa de rede de atenção: alerta', 'Detecção de mudança', 'Blocos de Corsi'],
                'Faixa etária', 'Radar de Métricas por Faixa Etária', 'radar_metrica_faixa')

    # Gráfico de violino bipartido
    gerar_violino_bipartido(df, 'Faixa etária', 'Rastreio de múltiplos objetos',
                            'Violino Bipartido de Rastreio de Objetos por Faixa Etária', 'violino_faixa_rastreio')

    # Testes t independentes entre Crianças e Adultos
    teste_t_independente(df, 'Tarefa de rede de atenção: alerta', 'Criança', 'Adulto')
    teste_t_independente(df, 'Detecção de mudança', 'Criança', 'Adulto')
    teste_t_independente(df, 'Blocos de Corsi', 'Criança', 'Adulto')

else:
    print("Não foi possível carregar os dados.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Gráfico salvo no Drive: /content/drive/My Drive/cochrane-et-al/regressao_alerta_visao_ordem3.png
Gráfico salvo no Drive: /content/drive/My Drive/cochrane-et-al/regressao_mudanca_distratores_ordem3.png
Gráfico salvo no Drive: /content/drive/My Drive/cochrane-et-al/regressao_corsi_rastreio_ordem2.png


<ipython-input-14-ccb3b6a6294b>:82: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x=x_col, y=y_col, data=df, palette="viridis")


Gráfico salvo no Drive: /content/drive/My Drive/cochrane-et-al/boxplot_faixa_mudanca.png
Gráfico salvo no Drive: /content/drive/My Drive/cochrane-et-al/radar_metrica_faixa.png


<ipython-input-14-ccb3b6a6294b>:128: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(x=x_col, y=y_col, data=df, split=True, inner="quart", palette="viridis")


Gráfico salvo no Drive: /content/drive/My Drive/cochrane-et-al/violino_faixa_rastreio.png
Teste t para Tarefa de rede de atenção: alerta entre Criança e Adulto: t=-0.0135, p=0.9893
Teste t para Detecção de mudança entre Criança e Adulto: t=-0.2835, p=0.7774
Teste t para Blocos de Corsi entre Criança e Adulto: t=-3.1911, p=0.0019
